# Facebook Prophet
## Breve Intro
<br>

Facebook Prophet è una libreria (R e Python *rilasciato sotto licenza MIT*) per lo studio di serie storiche univariate mediante [modelli GAM](https://en.wikipedia.org/wiki/Generalized_additive_model) sviluppata da Facebook. Potete trovare maggiori info sulla [pagina del progetto](https://facebook.github.io/prophet/).
<br>
Il modello additivo è una generalizzazione del modello di regressione lineare. Nello studio delle serie storiche il principio di base è sostituire la classica funzione lineare di una covariata con la funzione di smooth.
<br>
L’additività del modello consiste in una somma di tali funzioni, inoltre è più elastico dei classici modelli in quanto non impone rigide forme parametriche sulle funzioni ma ne fornisce una stima in modo iterativo.
<br>
Per installare prophet:
```
conda install -c conda-forge fbprophet
```
**N.B. pyforest è una libreria molto comoda che importa in una sola volta le librerie pandas, numpy, scipy, matplotlib e i rispettivi alias.**
<br>[Per maggiori info](https://pypi.org/project/pyforest/)

In [ ]:
from pyforest import *
from fbprophet import Prophet
%matplotlib inline

## Prophet su Airpassenger
<br>
Per prima cosa vediamo come si comporta Prophet su una serie mensile.
<br>

Airpassengers è una serie univariata, ha un trend lineare e un *effetto stagionalità* ***moltiplicativo.***

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv")
airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["#Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

Per prima cosa vanno cambiati i nomi delle variabili. Prophet vuole che i nomi siano prefissati:
* **ds**: la colonna del tempo
* **y**: la colonna dei valori

In [ ]:
airpassengers.rename(columns={"Month":"ds", "#Passengers":"y"}, inplace=True)

Dagli esempi che si trovano più facilmente online sembra che Prophet non abbia bisogno del setting dei parametri perchè fa tutto in automatico essendo rivolto in particolare ai *citizen data scientist*, poi scandagliando meglio si trovano molti **tricks & tips** interessanti.

In [ ]:
model = Prophet(growth="linear", n_changepoints=15, changepoint_range=0.5, yearly_seasonality=12, seasonality_mode="multiplicative", 
                changepoint_prior_scale=0.01, interval_width=0.95)

I parametri settati sono:
* growth: abbiamo lasciato il valore di default visto che il nostro dataset non presenta asintoti, in caso di un fenomeno che arriva a saturazione o ad un minimo si può utilizzare la "logistic" e impostare il limite (sia maggiore "cap" che minore "floor")
* changepoints: vogliamo la ricerca automatica, abbassiamo i valori di default, quindi cerchiamo 15 changepoints nel primo 50% della serie (sappiamo già che il trend è lineare quindi questo andrà a cercare dei sub-trend **sbagliando**)
* yearly_seasonality: la stagionalità stimata usando le serie di Fourier, solitamente il setting è su 'auto' ma vogliamo fare un test dando un valore
* seasonality_mode: sappiamo che l'effetto è moltiplicativo
* changepoint_prior_scale: di default a 0.05, più si avvicina a 0 più il trend diventa flessibile, abbiamo abbassato un pò il valore
* interval_width: solitamente gli intervalli sono all'80% noi stringiamo
<br>

Per maggiore approfondimento sui restanti parametri [consultare la pagina ufficiale](https://facebook.github.io/prophet/docs/quick_start.html)

In [ ]:
model.fit(airpassengers)

Come possiamo leggere la funzione 'auto' ha disabilitato la stagionalità settimanale e giornaliera avendo una granularità del dato mensile.
<br>
Ora creiamo un nuovo dataset su cui fare le previsioni con una apposita funzione di Prophet.

In [ ]:
future = model.make_future_dataframe(periods=24, freq='M', include_history=True)

I parametri:
* periods: quanti periodi voglimo predire
* freq: la frequenza di campionamento, in questo caso M perchè mensile, se fosse stato bimestrale '2M'.
* include_history: True crea un DataFrame che include le date del train più i periodi da predire

In [ ]:
forecast = model.predict(future)

Vediamo il DataFrame creato e successivamente usiamo la funzione plot di Prophet per avere un riscontro grafico.

In [ ]:
forecast.head()

In [ ]:
forecast.tail()

In [ ]:
model.plot(forecast)

Come possiamo vedere la nostra previsione "rasenta" l'overfitting.
<br>
Prophet è molto comodo per le serie con granularità sotto il giorno e alta frequenza di campionamento. Vediamo di seguito come si comporta con una serie storica estratta da un sensore della temperatura:
* campionamento a 10 minuti
* prenderemo solo una porzione del dataset, tre anni

In [ ]:
from tensorflow.keras.utils import get_file
import os

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
jena = pd.read_csv(csv_path)

In [ ]:
jena.head()

In [ ]:
temp = jena[["Date Time", "T (degC)"]].copy()
temp.rename(columns={"Date Time":"ds", "T (degC)":"y"}, inplace = True)
temp.shape

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(temp.shape[0]), temp["y"].values, color='tab:blue')
plt.gca().set(title="Jena Climate", xlabel="Time", ylabel="Temp")

Il campionamento è ogni 10 minuti quindi per avere 3 mesi 6 x 24 x 90 = 52560 osservazioni

In [ ]:
temp_3y = temp.iloc[:12960,:]

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(temp_3y.shape[0]), temp_3y["y"].values, color='tab:blue')
plt.gca().set(title="Jena Climate", xlabel="Time", ylabel="Temp")

Nel modello terrò quasi tutti i valori di default, tranne che per l'elasticità del trend, avendo una frequenza di campionamento alta verrà abbassato molto il valore del parametro.
<br>
Proveremo a predire 10 giorni.

In [ ]:
model_jena = Prophet(changepoint_prior_scale=0.01)

In [ ]:
model_jena.fit(temp_3y)

Una cosa che balza subito all'occhio è che ha effettivamente attivato la formula con effetto stagionale sia giornaliero che settimanale disabilitando quella annuale.

In [ ]:
future_jena = model_jena.make_future_dataframe(periods=1440, freq='10T')

In [ ]:
forecast_jena = model_jena.predict(future_jena)

In [ ]:
model_jena.plot(forecast_jena)

In [ ]:
12960+1440

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(1440), temp.loc[12960:14399,"y"].values, color='tab:blue')
plt.plot(range(1440), forecast_jena.loc[12960:,"yhat"].values, color='tab:red')
plt.gca().set(title="Jena Climate", xlabel="Time", ylabel="Temp")

Come si può vedere il modello sottostima il dato originale. Lascio a chi vuole provarci fare tuning del modello o preprocessing dei dati (che qui non abbiamo fatto).
<br>
Come sempre i feedback sono ben accetti!
### BUON LAVORO!